In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from collections import Counter

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.applications.convnext import preprocess_input, ConvNeXtTiny
from tensorflow.keras import layers, models, callbacks, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounte

In [2]:
print("--- Step 1: Loading and Preprocessing Data ---")
df = pd.read_csv("../dataset/train.csv")
print("train.csv loaded. Shape:", df.shape)
train_image_dir = "../dataset/train"
data = []
labels = []
IMG_SIZE = 224

for _, row in df.iterrows():
    img_path = os.path.join(train_image_dir, row['ID'])
    if os.path.exists(img_path):
        img = Image.open(img_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
        arr_img = preprocess_input(np.array(img))
        data.append(arr_img)
        labels.append(row['TARGET'])

data = np.array(data)
labels = np.array(labels)

print("\nData processed. Image array shape:", data.shape)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=len(le.classes_))
NUM_CLASSES = len(le.classes_)
print("Labels encoded successfully.")

--- Step 1: Loading and Preprocessing Data ---
train.csv loaded. Shape: (6400, 2)

Data processed. Image array shape: (6400, 224, 224, 3)
Labels encoded successfully.


In [5]:
def cutout(image_np):
    """
    Applies Cutout augmentation to a NumPy image.
    Erases a random square patch from the image.
    """
    try:
        height, width, _ = image_np.shape
        cutout_size = height // 4  # Size of the square to cut out (e.g., 25% of height)
        
        # Get random top-left corner for the cutout patch
        x = np.random.randint(0, width - cutout_size)
        y = np.random.randint(0, height - cutout_size)
        
        # Erase the patch by setting it to zero (black)
        image_np[y:y+cutout_size, x:x+cutout_size, :] = 0
    except Exception as e:
        # In case something goes wrong, just return the original image
        print(f"Warning: Cutout failed with error {e}")
    return image_np

In [6]:
print("\n--- Step 2: Splitting Data ---")
X_train, X_val, y_train, y_val = train_test_split(
    data, labels_categorical, test_size=0.2, random_state=42, stratify=labels_categorical
)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")

# --- 3. Setup Data Augmentation and tf.data.Dataset ---
print("\n--- Step 3: Setting up Data Augmentation and Dataset ---")
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    preprocessing_function=cutout
)
train_gen = datagen.flow(X_train, y_train, batch_size=32)
print("Data Augmentation generator created.")

def train_generator_wrapper():
    for x, y in train_gen:
        yield x, y

train_ds = tf.data.Dataset.from_generator(
    train_generator_wrapper,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
)
print("tf.data.Dataset created.")

class_weights_array = compute_class_weight('balanced', classes=np.unique(labels_encoded), y=labels_encoded)
class_weights = {i: w for i, w in enumerate(class_weights_array)}
print("Class Weights calculated:", class_weights)



--- Step 2: Splitting Data ---
Training set: 5120 samples
Validation set: 1280 samples

--- Step 3: Setting up Data Augmentation and Dataset ---
Data Augmentation generator created.
tf.data.Dataset created.
Class Weights calculated: {0: 4.0, 1: 4.0, 2: 0.8, 3: 0.8, 4: 0.8, 5: 0.8, 6: 0.8, 7: 0.8, 8: 0.8, 9: 0.8, 10: 0.8, 11: 0.8, 12: 4.0, 13: 4.0, 14: 0.8, 15: 0.8, 16: 0.8, 17: 0.8, 18: 4.0, 19: 0.8}


In [7]:
print("\n--- Step 4: Building the Model ---")
base_model = ConvNeXtTiny(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(1e-5)),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

f1_micro = metrics.F1Score(average='micro', name="f1_micro")
optimizer = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy', f1_micro])
print("Model compiled successfully.")
model.summary()



--- Step 4: Building the Model ---
Model compiled successfully.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 convnext_tiny (Functional)  (None, 7, 7, 768)         27820128  
                                                                 
 global_average_pooling2d_1  (None, 768)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense (Dense)               (None, 256)               196864    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                5140      
                                                                 
Total params: 28022132 (106.90 MB)
Trainable params: 2020

In [8]:
print("\n--- Step 5: Setting up Callbacks ---")
checkpoint = callbacks.ModelCheckpoint('convnext_tiny_model.keras', monitor='val_f1_micro', mode='max', save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(monitor='val_f1_micro', mode='max', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
print("Callbacks defined.")



--- Step 5: Setting up Callbacks ---
Callbacks defined.


In [20]:
print("\n--- Step 6: Starting Model Training ---")
steps_per_epoch = len(X_train) // 32
validation_steps = len(X_val) // 32

history = model.fit(
    train_ds,
    validation_data=(X_val, y_val),
    epochs=50,
    use_multiprocessing=True,   # <-- Make sure this is True
    workers=8,    
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, reduce_lr]
)
print("✅ Training complete!")


--- Step 6: Starting Model Training ---
Epoch 1/50
  4/160 [..............................] - ETA: 32:41 - loss: 1.2476 - accuracy: 0.7969 - f1_micro: 0.7969

KeyboardInterrupt: 

In [ ]:
print("\n--- Step 7: Evaluating Final Model on Validation Set ---")
val_loss, val_acc, val_f1 = model.evaluate(X_val, y_val)
print(f"✅ Final Validation Loss: {val_loss:.4f}")
print(f"✅ Final Validation Accuracy: {val_acc:.4f}")
print(f"✅ Final Validation F1 Micro: {val_f1:.4f}")
